# Test Distributional Shift
This notebook tests distributional shift for the dataset SQuAD.

## Imports

In [1]:
%%capture
!pip install transformers

In [2]:
%%capture
!pip install datasets

In [3]:
%%capture
!pip install nltk

In [4]:
%%capture
import nltk
nltk.download('punkt')

In [5]:
from transformers import BertTokenizer, BertModel, AlbertTokenizer, AlbertModel, DistilBertTokenizer, DistilBertModel
import torch
from datasets import load_dataset
from tqdm import tqdm
import matplotlib.pyplot as plt
from math import ceil

from nltk.tokenize import sent_tokenize, word_tokenize
import random
import numpy as np

## Loading and Preparing Texts

In [6]:
def remove_sentence(t, k=1):
    sentences = random.sample(sent_tokenize(t), k)
    text = t
    for s in sentences:
        text = text.replace(s, '')
        assert len(t) != len(text)
    return text, len(word_tokenize(s))

def remove_word(t, k=1):
    text = t
    for _ in range(k):
        words = word_tokenize(text)
        if not len([i for i,w in enumerate(words) if w.isalnum()]):
            break
        word_idx = random.choice([i for i,w in enumerate(words) if w.isalnum()])
        del words[word_idx]
        text = ''.join([(' ' if w.isalnum() else '')+w for w in words]).strip(' ')
    return text

In [7]:
dataset = load_dataset('squad', split='train')
print('Average number of sentences = {}'.format(np.mean([len(sent_tokenize(sample['context'])) for sample in dataset ])))

Dataset squad downloaded and prepared to /root/.cache/huggingface/datasets/squad/plain_text/1.0.0/1244d044b266a5e4dbd4174d23cb995eead372fbca31a03edc3f8a132787af41. Subsequent calls will reuse this data.
Average number of sentences = 5.1039509583442735


In [ ]:
N_WORD = 5
texts = []
dataset = load_dataset('squad', split='train')
for d in tqdm(dataset):
    if d['context'] not in texts:
        texts.append(d['context'])
random.shuffle(texts)        
n_samples = len(texts)
original = texts#random.sample(texts, n_samples)#texts[:n_samples]
word_removed = []
sentence_removed = []
for t in tqdm(original):
    sr, nw = remove_sentence(t)
    wr = remove_word(t, k=N_WORD)
    word_removed.append(wr)
    sentence_removed.append(sr)

In [ ]:
text_data = original+word_removed+sentence_removed
classes = [
    (n_samples, 'Original', 'tab:blue'),
    (n_samples, 'Word', 'tab:green'),
    (n_samples, 'Sentence', 'tab:red')
]

## Compare Distributional Shift
For different Classifiers (BERT, ALBERT, DistilBERT)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import numpy as np

In [ ]:
#def get_accuracy_rf(x_original, x_new):
#  X = torch.cat([x_original, x_new], dim=0).cpu().numpy()
#  y = np.array([0]*n_samples+[1]*n_samples)
#  X_train, X_test, y_train, y_test = train_test_split(X, y)
#  clf = RandomForestClassifier()
#  clf.fit(X_train, y_train)
#  print('Train : {}'.format(clf.score(X_train, y_train)))
#  print('Test : {}'.format(clf.score(X_test, y_test)))
def get_accuracy_rf(x_original, x_new):
    X = torch.cat([x_original, x_new], dim=0).cpu().numpy()
    y = np.array([0]*n_samples+[1]*n_samples)
    X_train, X_test, y_train, y_test = train_test_split(X, y)
    oob_score = 0
    best_clf = None
    for depth in [2, 5, 7, 10, 15, 20]:
        clf = RandomForestClassifier(max_depth=depth, oob_score=True)
        clf.fit(X_train, y_train)
        print('Depth {}, OOB score {}'.format(depth, clf.oob_score_))
        if clf.oob_score_>oob_score:
            oob_score = clf.oob_score_
            best_clf = clf
    print('--> Depth={}'.format(best_clf.max_depth))
    print('-'*20)
    train_acc = best_clf.score(X_train, y_train)
    test_acc = best_clf.score(X_test, y_test)
    print(X_train.shape)
    print('Train : {}({})'.format(train_acc, train_acc*(1-train_acc)/X_train.shape[0]))
    print('Test : {}({})'.format(test_acc, test_acc*(1-test_acc)/X_test.shape[0]))
    print('-'*20)

In [ ]:
def process_texts(texts, model, tokenizer):
    inputs = tokenizer(texts, return_tensors="pt", padding=True, truncation=True, max_length=512)
    return model(**(inputs.to(device)))[1].detach()

def batch(l, size):
    for i in range(0, len(l), size):
        yield l[i:i+size]

In [ ]:
def models_generator():
    model_classes = [
        BertModel,
        AlbertModel
    ]
    tokenizer_classes = [
        BertTokenizer,
        AlbertTokenizer
    ]
    model_names = [
        'bert-base-uncased',
        'albert-base-v2'
    ]
    for model_name, tokenizer_class, model_class in zip(model_names, tokenizer_classes, model_classes):
        tokenizer = tokenizer_class.from_pretrained(model_name)
        model = model_class.from_pretrained(model_name).to(device)
        yield model, tokenizer, model_name.split('-')[0].upper()

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
for model, tokenizer, name in models_generator():
    print('Model: {}'.format(name))
    results = []
    for b in batch(text_data, 5):
        results.append(process_texts(b, model, tokenizer))
    embeddings = torch.cat(results, 0)
    e_original = embeddings[:n_samples, :]
    e_word = embeddings[n_samples:2*n_samples, :]
    e_sentence = embeddings[2*n_samples:, :]
    print('Sentence vs Original')
    get_accuracy_rf(e_original, e_sentence)
    print('Word vs Original')
    get_accuracy_rf(e_original, e_word)
    print('\n\n')

Model: BERT
Sentence vs Original
Depth 2, OOB score 0.57520468661773
Depth 5, OOB score 0.5801453980801807
Depth 7, OOB score 0.5789455110107284
Depth 10, OOB score 0.5681818181818182
Depth 15, OOB score 0.5316205533596838
Depth 20, OOB score 0.515422077922078
--> Depth=5
--------------------
(28336, 768)
Train : 0.5983907396950875(8.48105810072888e-06)
Test : 0.5889265297480415(2.562905698782243e-05)
--------------------
Word vs Original
Depth 2, OOB score 0.7248729531338227
Depth 5, OOB score 0.7699040090344438
Depth 7, OOB score 0.7825734048560136
Depth 10, OOB score 0.7892433653303218
Depth 15, OOB score 0.7868788819875776
Depth 20, OOB score 0.7829616036137775
--> Depth=10
--------------------
(28336, 768)
Train : 0.898044889892716(3.231234670747303e-06)
Test : 0.7940927376667373(1.7309915482922258e-05)
--------------------



Model: ALBERT
Sentence vs Original
Depth 2, OOB score 0.610248447204969
Depth 5, OOB score 0.6200945793337098
Depth 7, OOB score 0.6223178994918125
Depth 10

In [ ]:
def process_texts_db(texts, model, tokenizer):
    inputs = tokenizer(texts, return_tensors="pt", padding=True, truncation=True, max_length=512)
    return model(**(inputs.to(device)))[0][:, 0, :].cpu().detach()

In [ ]:
model = DistilBertModel.from_pretrained('distilbert-base-uncased').to(device)
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

In [ ]:
print('Model: DistilBert')
results = []
for b in batch(text_data, 2):
    results.append(process_texts_db(b, model, tokenizer))
embeddings = torch.cat(results, 0)
e_original = embeddings[:n_samples, :]
e_word = embeddings[n_samples:2*n_samples, :]
e_sentence = embeddings[2*n_samples:, :]
print('Sentence vs Original')
get_accuracy_rf(e_original, e_sentence)
print('Word vs Original')
get_accuracy_rf(e_original, e_word)
print('\n\n')

Model: DistilBert
Sentence vs Original
Depth 2, OOB score 0.5481719367588933
Depth 5, OOB score 0.5525479954827781
Depth 7, OOB score 0.5351143421795596
Depth 10, OOB score 0.4772727272727273
Depth 15, OOB score 0.3782114624505929
Depth 20, OOB score 0.3268280632411067
--> Depth=5
--------------------
(28336, 768)
Train : 0.6231295878035008(8.287658971179417e-06)
Test : 0.555579081092526(2.6139208738610037e-05)
--------------------
Word vs Original
Depth 2, OOB score 0.7574463579898363
Depth 5, OOB score 0.7725861095426313
Depth 7, OOB score 0.7823616600790514
Depth 10, OOB score 0.7749858836815359
Depth 15, OOB score 0.7476002258610954
Depth 20, OOB score 0.7416713721061547
--> Depth=7
--------------------
(28336, 768)
Train : 0.8290866741953699(5.0007750164818775e-06)
Test : 0.7916578445903028(1.7460904265185137e-05)
--------------------



